## Zadanie domowe - Algorytm Canny'ego

Celem zadania domowego jest wykonanie pełnej implementacji algorytmu Canny'ego.

W ramach ćwiczenia w trakcie laboratorium wyznaczono obrazy $g_{NH}$ i $g_{NL}$.
Dla przypomnienia:
Można powiedzieć, że na obrazie $g_{NH}$ są "pewne" krawędzie.
Natomiast na $g_{NL}$ "potencjalne".
Często krawędzie "pewne" nie są ciągłe.
Wykorzystuje się więc krawędzie "potencjalne", aby uzupełnić nieciągłości.
Procedura wygląda następująco:
1. Stwórz stos zawierający wszystkie piksele zaznaczone na obrazie $g_{NH}$.
W tym celu wykorzystaj listę współrzędnych `[row, col]`.
Do pobrania elementu z początku służy metoda `list.pop()`.
Do dodania elementu na koniec listy służy metoda `list.append(new)`.
2. Stwórz obraz, który będzie zawierał informację czy dany piksel został już odwiedzony.
3. Stwórz obraz, który zawierać będzie wynikowe krawędzie.
Jej rozmiar jest równy rozmiarowi obrazu.
4. Wykonaj pętlę, która będzie pobierać elementy z listy, dopóki ta nie będzie pusta.
W tym celu najlepiej sprawdzi się pętla `while`.
    - W każdej iteracji pobierz element ze stosu.
    - Sprawdź, czy dany element został już odwiedzony.
    - Jeśli nie został, to:
        - Oznacz go jako odwiedzony,
        - Oznacz piksel jako krawędź w wyniku,
        - Sprawdź otoczenie piksela w obrazie $g_{NL}$,
        - Dodaj do stosu współrzędne otoczenia, które zawierają krawędź.
        Można to wykonać np. pętlą po stworzonym otoczeniu.
7. Wyświetl obraz oryginalny, obraz $g_{NH}$ oraz obraz wynikowy.
8. Porównaj wynik algorytmu z wynikiem OpenCV.

Pomocnicze obrazy $g_{NH}$ i $g_{NL}$ zostały wprowadzone dla uproszczenia opisu.
Algorytm można zaimplementować w bardziej "zwarty" sposób.

Na podstawie powyższego opisu zaimplementuj pełny algorytm Canny'ego.

In [ ]:
import cv2
from matplotlib import pyplot as plt
import numpy as np
import math
import os

if not os.path.exists("dom.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/09_Canny/dom.png --no-check-certificate

house = cv2.imread('dom.png', cv2.IMREAD_GRAYSCALE)


def nonmax(alfa_kw, Mxy):

    X,Y = alfa_kw.shape
    gn=np.zeros((X,Y))
    
    for i in range (1,X-1):
        for j in range (1,Y-1):

            if alfa_kw[i,j] == 0:
                if Mxy[i,j-1]>Mxy[i,j] or Mxy[i,j+1]>Mxy[i,j]:
                    gn[i,j]=0
                else:
                  gn[i,j]=Mxy[i,j]
  
            elif alfa_kw[i,j] == 45:
                if Mxy[i+1,j-1]>Mxy[i,j] or Mxy[i-1,j+1]>Mxy[i,j]:
                    gn[i,j]=0
                else:
                  gn[i,j]=Mxy[i,j]

            elif alfa_kw[i,j] == 90 :
                if Mxy[i+1,j]>Mxy[i,j] or Mxy[i-1,j]>Mxy[i,j]:
                    gn[i,j]=0
                else:
                  gn[i,j]=Mxy[i,j]

            elif alfa_kw[i,j] == 135:
                if Mxy[i-1,j-1]>Mxy[i,j] or Mxy[i+1,j+1]>Mxy[i,j]:
                    gn[i,j]=0
                else:
                  gn[i,j]=Mxy[i,j]                         
    return gn

def canny1 (house, tl, th):

  house_blur = cv2.GaussianBlur(house, (3, 3), 0)

  gx = cv2.Sobel(house_blur,cv2.CV_64F,1,0,ksize=5)  
  gy = cv2.Sobel(house_blur,cv2.CV_64F,0,1,ksize=5)  
  Mxy = np.sqrt(gx*gx + gy*gy)
  alfa = np.arctan2(gy, gx)
  alfa = alfa * 180./np.pi

  X, Y = house_blur.shape
  alfa_kw = np.zeros((X,Y))


  for i in range (X):
    for j in range (Y):

      if alfa[i,j] > -22.5 and alfa[i,j] < 22.5:
        alfa_kw[i,j] = 0
        #poziomy d2

      elif alfa[i,j] > 22.5 and alfa[i,j] < 67.5:
        alfa_kw[i,j] = 45
        #skośny prawy d3

      elif alfa[i,j] > 67.5 and alfa[i,j] < 112.5:
        alfa_kw[i,j] = 90
        #pionowy d1

      elif alfa[i,j] > 112.5 and alfa[i,j] < 157.5:
        alfa_kw[i,j] = 135
        #skośny lewy d4

      elif alfa[i,j] > 157.5 or alfa[i,j] < -157.5:
        alfa_kw[i,j] = 180

      elif alfa[i,j] > -157.5 and alfa[i,j] < -112.5:
        alfa_kw[i,j] = 225

      elif alfa[i,j] > -112.5 and alfa[i,j] < -67.5:
        alfa_kw[i,j] = 270

      elif alfa[i,j] > -67.5 and alfa[i,j] < -22.5:
        alfa_kw[i,j] = 315

  gn = nonmax(alfa_kw, Mxy)
  gnh = np.where(gn >= th, 1, 0)

  gnl1 = np.where(gn < th, 1, 0)
  gnl2 = np.where(gn >= tl, 1, 0)
  gnl = gnl1 * gnl2



  return gnh, gnl


In [ ]:
def canny2 (gnh, gnl):
  X,Y = gnh.shape
  stack = []
  for i in range (1, X-1):
    for j in range (1, Y-1):
      if gnh[i,j] != 0:
        stack.append([i,j])

  visited = np.zeros((X,Y))
  result_edges = np.zeros((X,Y))

  while stack:
    el = stack.pop()

    if visited[el[0], el[1]] == 0:
       visited[el[0], el[1]] = 1
       result_edges[el[0], el[1]] = 1

       for i in range(el[0]-1,el[0] + 2):
         for j in range(el[1]-1,el[1] + 2):
           if gnl[i,j] != 0:
              stack.append([i,j])

  return result_edges

gnh,gnl = canny1(house,4,8)
results = canny2(gnh,gnl)

In [ ]:

fig, axs = plt.subplots(1, 3)

fig.set_size_inches(17, 17)
axs[0].imshow(house, 'gray')
axs[0].axis('off')
axs[0].set_title("obraz orginalny")

axs[1].imshow(gnh, 'gray',vmin = 0, vmax = 1)
axs[1].axis('off')
axs[1].set_title("gnh")

axs[2].imshow(results, 'gray', vmin = 0, vmax = 1)
axs[2].axis('off')
axs[2].set_title("pełna implementacja")

fig.show()